In [161]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import os
from time import time
from pprint import pprint
import collections

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
sns.set(font_scale=1.3)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

np.random.seed(37)

In [287]:
df_model = pd.read_pickle('../../data/final/NotIot_clean_EN_news_lexic.p')

In [288]:
df_model.head()

,count_words,count_capital_words,count_excl_quest_marks,sentiment,origin_text,clean_text,lex_anger,lex_sadness,lex_joy,lex_fear
0,8,0,0,joy,Test to predict breast cancer relapse is approved,test predict breast cancer relapse approved,0.096167,0.234167,0.081,0.2240
1,8,0,0,sadness,"Two Hussein allies are hanged, Iraqi official says",two hussein allies hanged iraqi official says,0.000000,0.000000,0.000,0.0000
2,5,1,0,joy,Sights and sounds from CES,sights sounds ces,0.000000,0.000000,0.000,0.0000
3,6,0,0,joy,Schuey sees Ferrari unveil new car,schuey sees ferrari unveil new car,0.000000,0.000000,0.000,0.0000
4,8,0,0,fear,Closings and cancellations top advice on flu outbreak,closings cancellations top advice flu outbreak,0.000000,0.073000,0.000,0.1745


In [289]:
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('sentiment', axis=1), df_model.sentiment, test_size=0.1, random_state=37)

In [290]:
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X, **transform_params):
        return X[self.cols]

    def fit(self, X, y=None, **fit_params):
        return self

In [291]:
def grid_vect(clf, parameters_clf, X_train, X_test, parameters_text=None, vect=None):
    
    textcountscols = ['count_capital_words','count_emojis'
                      ,'count_excl_quest_marks','count_hashtags','count_mentions','count_urls',
                      'count_words']
    
    lexiccols = ['lex_anger','lex_sadness','lex_joy','lex_fear']#,'lex_surprise','lex_disgust','lex_neutre']
    
    features = FeatureUnion([# ('textcounts', ColumnExtractor(cols=textcountscols)),
                           # ('lexiccols', ColumnExtractor(cols=lexiccols)),
                        ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text')), ('vect', vect)]))]
                            , n_jobs=-1)

    
    pipeline = Pipeline([
        ('features', features)
        , ('clf', clf)
    ])
    
    # Join the parameters dictionaries together
    parameters = dict()
    if parameters_text:
        parameters.update(parameters_text)
    parameters.update(parameters_clf)

    # initiate gridsearchCV with parameters and pipline
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=5)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best CV score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    print("Test score with best_estimator_: %0.3f" % grid_search.best_estimator_.score(X_test, y_test))
    print("\n")
    print("Classification Report Test Data")
    print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

    print("all results")
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    params = grid_search.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    
    return grid_search

In [292]:
# Parameter grid settings for the vectorizers (Count and TFIDF)
parameters_vect = {
    'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
    'features__pipe__vect__ngram_range': ((1, 1), (1, 2)),
    'features__pipe__vect__min_df': (1,2)
}

# Parameter grid settings for MultinomialNB
parameters_mnb = {
    'clf__alpha': (0.25, 0.5, 0.75)
}

In [293]:
mnb = MultinomialNB()

In [294]:
countvect = CountVectorizer()

In [295]:
# MultinomialNB
best_mnb_countvect = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=countvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   26.9s finished


done in 27.714s

Best CV score: 0.510
Best parameters set:
	clf__alpha: 0.75
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 1
	features__pipe__vect__ngram_range: (1, 2)
Test score with best_estimator_: 0.424


Classification Report Test Data
              precision    recall  f1-score   support

       anger       1.00      0.07      0.13        14
        fear       0.37      0.47      0.41        15
         joy       0.47      0.74      0.58        35
     sadness       0.38      0.32      0.34        19
    surprise       0.25      0.12      0.17        16

   micro avg       0.42      0.42      0.42        99
   macro avg       0.49      0.34      0.33        99
weighted avg       0.48      0.42      0.38        99

all results
0.492099 (0.040622) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 1)}
0.484199 (0.035019) with: {'clf__alpha': 0.25, 'features__pipe__vect__ma

In [296]:
tfidfvect = TfidfVectorizer()

In [297]:
best_mnb_tfidf = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=tfidfvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   20.0s finished


done in 20.460s

Best CV score: 0.517
Best parameters set:
	clf__alpha: 0.25
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 1
	features__pipe__vect__ngram_range: (1, 1)
Test score with best_estimator_: 0.404


Classification Report Test Data
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        14
        fear       0.33      0.40      0.36        15
         joy       0.47      0.74      0.58        35
     sadness       0.32      0.32      0.32        19
    surprise       0.29      0.12      0.17        16

   micro avg       0.40      0.40      0.40        99
   macro avg       0.28      0.32      0.29        99
weighted avg       0.32      0.40      0.35        99

all results
0.516930 (0.023105) with: {'clf__alpha': 0.25, 'features__pipe__vect__max_df': 0.25, 'features__pipe__vect__min_df': 1, 'features__pipe__vect__ngram_range': (1, 1)}
0.507901 (0.015478) with: {'clf__alpha': 0.25, 'features__pipe__vect__ma

In [11]:
""""#max_df: 0.25 or maximum document frequency of 25%.
#min_df: 2 or the words need to appear in at least 2 tweets
#ngram_range: (1, 1)
#clf__alpha: 0.75
textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions','count_urls','count_words']
    
features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                         , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text'))
                                              , ('vect', CountVectorizer(max_df=0.25, min_df=2, ngram_range=(1,1)))]))]
                       , n_jobs=-1)

pipeline = Pipeline([
    ('features', features)
    , ('clf', MultinomialNB(alpha=0.75))
])

best_model = pipeline.fit(df_model.drop('sentiment', axis=1), df_model.sentiment)"""""

'"#max_df: 0.25 or maximum document frequency of 25%.\n#min_df: 2 or the words need to appear in at least 2 tweets\n#ngram_range: (1, 1)\n#clf__alpha: 0.75\ntextcountscols = [\'count_capital_words\',\'count_emojis\',\'count_excl_quest_marks\',\'count_hashtags\'\n                      ,\'count_mentions\',\'count_urls\',\'count_words\']\n    \nfeatures = FeatureUnion([(\'textcounts\', ColumnExtractor(cols=textcountscols))\n                         , (\'pipe\', Pipeline([(\'cleantext\', ColumnExtractor(cols=\'clean_text\'))\n                                              , (\'vect\', CountVectorizer(max_df=0.25, min_df=2, ngram_range=(1,1)))]))]\n                       , n_jobs=-1)\n\npipeline = Pipeline([\n    (\'features\', features)\n    , (\'clf\', MultinomialNB(alpha=0.75))\n])\n\nbest_model = pipeline.fit(df_model.drop(\'sentiment\', axis=1), df_model.sentiment)'

In [12]:
##df_model_pos = pd.read_pickle('../data/df_model_pos.p')
#best_model.predict(df_model_pos).tolist()

In [13]:
#df_model_neg = pd.read_pickle('../data/df_model_neg.p')
#best_model.predict(df_model_neg).tolist()